In [2]:
"""
hier wird rnn mit class torch.nn.RNN implementiert
"""
from pathlib import Path
import torch  
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using '{device}' device")

class RNN(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first= True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x, future_steps = 1):
        """
        Forward Pass
        x: (batch_size, sequence_length, input_size)
        future_steps: Number of future steps to predict
        out: batch_size, sequence_length, hidden_size
        hidden_state : num_layers, batch_size, hidden_size
        """ 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        outputs = []
        out, h = self.rnn(x, h0) # pass trough RNN

        out = self.fc(out[:, -1, :]) # initial output from last time step, Extrahiert den Hidden State des letzten Zeitschritts der Sequenz für jeden Batch.
        outputs.append(out)
        
        for _ in range(future_steps - 1): # iterative prediction for future steps
            out, h = self.rnn(out.unsqueeze(1), h)
            out = self.fc(out[:, -1, :])
            outputs.append(out)

        return torch.stack(outputs, dim=1) # combine output : (batch_size, future_steps, num_classes)
     
# Custom Dataset Class
class TimeSeriesDataset(Dataset):
    def __init__(self, file_path, sequence_length, future_steps):
        """
        Prepares data for sequence-based training with future steps targets
        """
        
        # Drop the time column (assume it's not needed for the model)
        data = pd.read_csv(file_path).iloc[1:, 1:].values  # Load X, Y, Z
        
        #convert to Pytorch tensor
        data = torch.tensor(data,dtype=torch.float32)

        #compute mean and standard deviation over features (columns)
        mean = data.mean(dim=0)
        std = data.std(dim=0)

         # Apply normalization
        data = (data - mean) / std
        

        self.sequence_length = sequence_length
        self.future_steps = future_steps

        self.X, self.Y = [], []
       
        for i in range(len(data) - sequence_length - future_steps + 1):
            self.X.append(data[i:i+sequence_length])
            self.Y.append(data[i+sequence_length: i+sequence_length+future_steps])     

        # Create inputs (X) and targets (Y)
        self.X = torch.stack(self.X)  # All rows except the last
        self.Y = torch.stack(self.Y)   # All rows except the first (shifted)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        # Return a single sample (input, target)
        return self.X[index], self.Y[index]
        

def create_dataloader(file_path, batch_size, sequence_length, future_steps, shuffle=False):
    """
    Creates a DataLoader from the given file path.
    """
    dataset = TimeSeriesDataset(file_path, sequence_length, future_steps)

    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=True)



# Train Function
def train(model, data, epochs, optimizer, loss_fn, future_steps):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()
    train_losses = {}
    
    print("=> Starting training")

    for epoch in range(epochs):
        
        epoch_losses = list()
        
        for X, Y in data:  # X, Y are batches
            

            # Send tensors to the device
            X, Y = X.to(device), Y.to(device)

            # Clear gradients
            optimizer.zero_grad()

            prediction = model(X, future_steps = future_steps)
            loss = loss_fn(prediction, Y)
            # prediction of shape (batch_size, future_steps, output_size=features)
            # Y of shape (batch_size, future_steps, output_size=features)
           

            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())

            
        # Store epoch loss
        train_losses[epoch] = np.mean(epoch_losses) # mean value for all batches in one epoch
        if (epoch +1) % 10 == 0: 
            print(f"=> Epoch: {epoch + 1}/{epochs}, Loss: {train_losses[epoch]:.4f}") 

    return train_losses

def save_model(model, file_name):
    torch.save(model.state_dict(), file_name)
    print(f"Model saved to {file_name}")


def save_model(model, file_name):
    torch.save(model.state_dict(), file_name)
    print(f"Model saved to {file_name}")

def evaluate_and_plot(model, dataloader, mean, std):
    model.eval()
    predictions, actuals = [], []

    with torch.no_grad():
        for X, Y in dataloader:
            X, Y = X.to(device), Y.to(device)
            output = model(X, future_steps=Y.size(1))
            output = output * std + mean  # Denormalize predictions
            Y = Y * std + mean  # Denormalize actuals
            predictions.append(output.cpu().numpy())
            actuals.append(Y.cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)

    plt.figure(figsize=(15, 10))
    for i in range(predictions.shape[-1]):
        plt.subplot(predictions.shape[-1], 1, i + 1)
        plt.plot(predictions[:, :, i].flatten(), label=f'Predicted Feature {i + 1}', linestyle='--')
        plt.plot(actuals[:, :, i].flatten(), label=f'Actual Feature {i + 1}')
        plt.xlabel('Time Steps')
        plt.ylabel(f'Feature {i + 1}')
        plt.legend()

    plt.tight_layout()
    plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [1]:
### Evaluation


if __name__ == "__main__":
    # Configuration
    model_path = "/path/to/rnn_model.pth"  # Change this to your model path
    file_path = "/path/to/test_dataset.csv"  # Change this to your dataset path
    sequence_length = 60
    future_steps = 20
    batch_size = 1
    input_size = 3
    hidden_size = 80
    num_layers = 2
    output_size = 3

    # Define mean and std (adapt these values to your data)
    mean = torch.tensor([mean_x, mean_y, mean_z])  # Replace with actual means
    std = torch.tensor([std_x, std_y, std_z])  # Replace with actual std devs

    # Load model
    model = RNN(input_size, hidden_size, num_layers, output_size).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # Create dataloader
    dataloader = create_dataloader(file_path, batch_size, sequence_length, future_steps, mean, std)

    # Evaluate and plot
    evaluate_and_plot(model, dataloader, mean, std)

NameError: name 'torch' is not defined

In [ ]:
# Training

